In [2]:
import pandas as pd
import seaborn as sns
sns.set() 
%matplotlib inline 
pd.set_option('max_columns',999)

In [ ]:
# PAYMENT
pay_data = pd.read_csv("data//PANEL//PAYMENT data.csv", engine = "python")
pay_brand = pd.read_csv("data//PANEL//PAYMENT Summary_Brand.csv", engine = "python")
pay_cat = pd.read_csv("data//PANEL//PAYMENT Summary_Brand.csv", engine = "python")
pay_total = pd.read_csv("data//PANEL//PAYMENT Summary_Total.csv", engine = "python")

In [6]:
# PPDB
data_panel = pd.read_csv("data//PANEL//PPDB data.csv", engine = "python")

In [ ]:
# APP
app_time = pd.read_csv("data//PANEL//APP Refine_AppTime.csv", engine = "python")
app_cat_g = pd.read_csv("data//PANEL//APP Refine_CategoryTime_G.csv", engine = "python")
app_cat_mme = pd.read_csv("data//PANEL//APP Refine_CategoryTime_MME.csv", engine = "python")
app_sum_time = pd.read_csv("data//PANEL//APP Refine_Usage_Sum_Time.csv", engine = "python")

---

# 이항변수화


## PAYMENT 데이터 이항변수화

In [20]:
# 이항변수화 하고자 하는 데이터프레임 넣으면 해결해주는 함수!

def Func2(df, col):
    valist = df[col].unique()
    t = pd.DataFrame(df.groupby("PANEL_ID")[col].apply(list)).reset_index()
    for i in range(len(valist)):
        t[col+"_"+valist[i]] = [1 if valist[i] in x else 0 for x in t[col]]
    return t.drop(col, axis = 1)

In [21]:
# PAY_APPROVAL_TYPE
# L, F

pay_data_1 = Func2(pay_data, "PAY_APPROVAL_TYPE")

In [22]:
# PAY_CATEGORY_CODE
# int형 변수 -> string으로 

pay_data["PAY_CATEGORY_CODE_"] = [str(x) for x in pay_data["PAY_CATEGORY_CODE"]]
pay_data_2 = Func2(pay_data, "PAY_CATEGORY_CODE_")

In [23]:
# PAY_SMS_REGISTRATION_MONTH

pay_data_3 = Func2(pay_data, "PAY_SMS_REGISTRATION_MONTH")

In [24]:
# PAY_SMS_REGISTRATION_DATE

pay_data_4 = Func2(pay_data, "PAY_SMS_REGISTRATION_DATE")

In [25]:
# PAY_SMS_REGISTRATION_TIME
# int형 변수 -> string으로 

pay_data["PAY_SMS_REGISTRATION_TIME_"] = [str(x) for x in pay_data["PAY_SMS_REGISTRATION_TIME"]]
pay_data_5 = Func2(pay_data, "PAY_SMS_REGISTRATION_TIME_")

In [26]:
# PAY_CARD_PAYMENT_TYPE

pay_data_6 = Func2(pay_data, "PAY_CARD_PAYMENT_TYPE")

In [27]:
# PAY_APPROVAL_METHOD
# int형 변수 -> string으로 

pay_data["PAY_APPROVAL_METHOD_"] = [str(x) for x in pay_data["PAY_APPROVAL_METHOD"]]
pay_data_7 = Func2(pay_data, "PAY_APPROVAL_METHOD_")

In [28]:
# PAY_PRICE
# 각 사람별 평균 구매액 구하기
pay_data_8 = pd.DataFrame(pay_data.groupby("PANEL_ID").mean()["PAY_PRICE"].apply(int)).reset_index()

# 사분위수 구하기
print(pay_data_8.quantile([0.25, 0.5 , 0.75] ))

# 가격 범위 나누기
pay_data_8["PAY_PRICE_0"] = [1 if x< 25609.5 else 0 for x in pay_data_8["PAY_PRICE"]]
pay_data_8["PAY_PRICE_1"] = [1 if 25609.5<=x< 61144 else 0 for x in pay_data_8["PAY_PRICE"]]
pay_data_8["PAY_PRICE_2"] = [1 if 61144<=x< 139590.5 else 0 for x in pay_data_8["PAY_PRICE"]]
pay_data_8["PAY_PRICE_3"] = [1 if 139590.5<=x else 0 for x in pay_data_8["PAY_PRICE"]]
pay_data_8 = pay_data_8.drop("PAY_PRICE", axis = 1)
pay_data_8 = pay_data_8.applymap(lambda x: str(x))

      PAY_PRICE
0.25    25609.5
0.50    61144.0
0.75   139590.5


In [29]:
# 전체 데이터 합치기

# 각 데이터 row 수 동일한지 확인
print(pay_data_1.shape, pay_data_2.shape, pay_data_3.shape, pay_data_4.shape, pay_data_5.shape, pay_data_6.shape, pay_data_7.shape, pay_data_8.shape)

# row수 : panel수
pay_all  = pd.merge(pay_data_1, pay_data_2, on = "PANEL_ID")
pay_all = pd.merge(pay_all, pay_data_3, on = "PANEL_ID")
pay_all = pd.merge(pay_all, pay_data_4, on = "PANEL_ID")
pay_all = pd.merge(pay_all, pay_data_5, on = "PANEL_ID")
pay_all = pd.merge(pay_all, pay_data_6, on = "PANEL_ID")
pay_all = pd.merge(pay_all, pay_data_7, on = "PANEL_ID")
pay_all = pd.merge(pay_all, pay_data_8, on = "PANEL_ID")

# 합쳐진 데이터 shape 확인
pay_all.shape

(11187, 3) (11187, 64) (11187, 2) (11187, 31) (11187, 25) (11187, 6) (11187, 21) (11187, 5)


(11187, 150)

In [30]:
# pay_dummies.csv

pay_all.to_csv("data/0109_payment_dummies.csv")